In [1]:
import os
import pathlib
import numpy as np
import datetime as dt
import pandas as pd
from jupyter_dash import JupyterDash
from dash import html,dcc
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import calendar
from dash.exceptions import PreventUpdate
from dash.dependencies import Input, Output, State

In [2]:
data = pd.read_csv("D202.csv")

#Data Preprocessing
data["COST"] = pd.to_numeric(data["COST"].str.slice(1,5))
data['DATE'] = pd.to_datetime(data['DATE'])
data['HOUR'] = pd.to_datetime(data['START TIME'],format ="%H:%M").dt.hour
data['YEAR'] = data['DATE'].dt.year
data['MONTH'] = data['DATE'].dt.to_period('M')
data['Time_Of_Day'] = data['HOUR'].apply(lambda x: "AM" if (x<12) else "PM")

#new dfs
daily_cost = data.groupby("DATE",sort=False)["COST"].sum().reset_index()
cost_by_time_of_day = data.groupby(["Time_Of_Day","YEAR"])['COST'].sum().reset_index()
monthly_cost = data.groupby(data['DATE'].dt.month)["COST"].sum().reset_index().rename(columns= dict(DATE = "MONTH"))
monthly_cost['MONTH'] = monthly_cost['MONTH'].apply(lambda x: calendar.month_abbr[x])

In [3]:
#BarPlot
fig1 = px.bar(cost_by_time_of_day,x="YEAR",y="COST",color = "Time_Of_Day",barmode="group",template= "plotly_dark",
                    width=620,
                    height = 400,
                    title="Energy Cost During the Day",
                    labels=dict(Time_Of_Day = 'Time of Day'),
                    category_orders=dict(Time_Of_Day = ['AM',"PM"]),
                    color_discrete_map=dict(
                    AM = "#FFDEAD", PM = "#27346F")
                    )
fig1.update_layout(yaxis_tickformat = '$')
fig1.add_shape(type='line',name ="Average",line_color = 'white',line_width = 1.5,line_dash='dash', x0=0,x1=1,xref="paper",y0=np.mean(cost_by_time_of_day['COST']),y1=np.mean(cost_by_time_of_day['COST'])) 
fig1.add_annotation(
    dict(
        x= cost_by_time_of_day['YEAR'][0],
        y=np.mean(cost_by_time_of_day['COST']),
        text='Average Cost',
        hovertext = f"Average Cost: {round(np.mean(cost_by_time_of_day['COST']),2)} $ <br>"
   )) 

In [4]:
#Area-Chart
fig2 = px.area(daily_cost,x="DATE",y="COST",title="Energy Cost - Daily",width = 1000,height = 800,markers = True,line_shape = 'spline',template= "plotly_dark")
fig2.update_traces(line=dict(color = '#27346F'),marker =dict(size = 1.8,color = 'white',symbol =100) )
fig2.update_xaxes(rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ]),font = dict(color = 'black')
        ),
        rangeslider=dict(
            visible=True
        ),type = 'date')

fig2.update_layout(yaxis_tickformat = '$')
fig2.add_trace(go.Scatter(x=[daily_cost['DATE'][0],daily_cost['DATE'].max()],y=([np.mean(daily_cost['COST'])]*10),name = 'Average Cost',line =dict(dash ='dash',color = 'white'),mode = 'lines',
     hovertemplate= 
        "Date: %{x|%Y/%m/%d}<br>" +
        f"Average Cost: {round(np.mean(daily_cost['COST']),2)} <br>" +
        "<extra></extra>",
        hoveron = 'points+fills',
        ))

In [5]:
#Pie
fig3 = px.pie(monthly_cost,values = "COST",names ='MONTH',labels = 'MONTH',title ="Cost Percentage by Month",hole=0.73,opacity=.97,color_discrete_sequence=px.colors.cyclical.Twilight,template ='plotly_dark',height=420,width = 620)
fig3.update_traces(textposition='inside',textinfo='percent+label',
                 hovertemplate = "Month: %{label} <br>Cost: %{value}$ </br>")

In [6]:
#Dash Layout
app = JupyterDash(__name__)
app.title = "Energy Cost"

app.layout = html.Div([
        html.Div([
            html.H1('Energy Cost Dashboard')],
            className="app__header"),
        html.Div([
            html.Div(children=[
                dcc.Graph(id = 'Areachart',
                figure = fig2)],
                className="columns wind__speed__container"),
            html.Div([
                html.Div([
                    dcc.Graph(id = 'Barplot',
                    figure = fig1)
                ],className="graph__container first",),
                html.Div([
                    dcc.Graph(id = 'Piechart',
                    figure = fig3)
                ],className="graph__container second")
            ],className="two-third columns histogram__direction")
        ],className="app__content")
],className="app__container")

In [7]:
if __name__ == "__main__":
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/
